### Base

In [1]:
import os
import random
import argparse
from time import perf_counter

import cv2
import tqdm
import numpy as np
import pandas as pd
from PIL import Image
from numpy import random
import matplotlib.pyplot as plt
from IPython.display import display, Image, clear_output

import torch
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset

from ultralytics import YOLO # yolov8

from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

import warnings
warnings.filterwarnings("ignore")

c:\Users\nguye\anaconda3\envs\ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
devices = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
print(f"Using {devices} device")

Using cuda:0 device


In [3]:
!nvidia-smi

Sat Apr 20 10:40:45 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P0             10W /   60W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
!yolo checks



Ultralytics YOLOv8.2.2 🚀 Python-3.12.2 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)
Setup complete ✅ (12 CPUs, 31.3 GB RAM, 279.1/474.3 GB disk)

OS                  Windows-11-10.0.22631-SP0
Environment         Windows
Python              3.12.2
Install             pip
RAM                 31.34 GB
CPU                 AMD Ryzen 5 5600H with Radeon Graphics
CUDA                12.1

matplotlib          ✅ 3.8.4>=3.3.0
opencv-python       ✅ 4.9.0.80>=4.6.0
pillow              ✅ 10.2.0>=7.1.2
pyyaml              ✅ 6.0.1>=5.3.1
requests            ✅ 2.31.0>=2.23.0
scipy               ✅ 1.13.0>=1.4.1
torch               ✅ 2.2.2+cu121>=1.8.0
torchvision         ✅ 0.17.2+cu121>=0.9.0
tqdm                ✅ 4.66.2>=4.64.0
psutil              ✅ 5.9.0
py-cpuinfo          ✅ 9.0.0
thop                ✅ 0.1.1-2209072238>=0.1.1
pandas              ✅ 2.2.2>=1.1.4
seaborn             ✅ 0.13.2>=0.11.0


### Load OpenSource Dataset

In [5]:
from roboflow import Roboflow

rf = Roboflow(api_key="API_here")

In [6]:

# Link: https://universe.roboflow.com/firedetection-cqgz2/firedetectyolov7/dataset/2
fire1_project = rf.workspace("firedetection-cqgz2").project("firedetectyolov7")
fire1_dataset = fire1_project.version(2).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.2, to fix: `pip install ultralytics==8.0.196`


In [9]:
# Link: https://universe.roboflow.com/agostino-abbatecola-52ty4/veryfiresmokedetection/dataset/3
fire2_project = rf.workspace("agostino-abbatecola-52ty4").project("veryfiresmokedetection")
fire2_dataset = fire2_project.version(3).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [10]:
# Link: https://universe.roboflow.com/atlantic-technological-university/wildfire-v2/dataset/5
fire3_project = rf.workspace("atlantic-technological-university").project("wildfire-v2")
fire3_dataset = fire3_project.version(5).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [11]:
# Link: https://universe.roboflow.com/spyrobot/fire-smoke-and-human-detector/dataset/31
fire4_project = rf.workspace("spyrobot").project("fire-smoke-and-human-detector")
fire4_dataset = fire4_project.version(31).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


### Support Functions

In [9]:
# Constants
MAX_COS_DIST = 0.5
MAX_TRACK_AGE = 70
deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
frames = []
unique_track_ids = set()

In [17]:
def track_object(input_vid: str, save_path: str):
    """
    Main function to track object in the input video
        1. Reads images from an input video stream
        2. Get detections of people in the input frame using YOLO
        3. Processes the detections along with previous tracks using DeepSORT
        4. Each output frame with refined bounding boxes to an output video stream

    Args:
    - input_vid: str, path to the input video
    - save_path: str, path to save the output video
    """
    global devices, MAX_COS_DIST, MAX_TRACK_AGE, deep_sort_weights, tracker, frames, unique_track_ids
    
    # Initialize tracker
    tracker = DeepSort(model_path=deep_sort_weights, max_age=MAX_TRACK_AGE)

    cap = cv2.VideoCapture(input_vid)
    
    model = YOLO(r"your/model/location/here")
    print(model.names)

    # Get the video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(save_path, fourcc, fps, (frame_width, frame_height))

    # core processing loop
    i = 0
    counter, fps, elapsed = 0, 0, 0
    start_time = perf_counter()
    
    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            
            og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = og_frame.copy()

            model = YOLO(r"your/model/location/here")

            results = model.predict(frame, devices, conf = 0.8, classes = None)

            class_names = ['fire', 'smoke']

            for result in results:
                boxes = result.boxes  # Boxes object for bbox outputs
                probs = result.probs  # Class probabilities for classification outputs
                cls = boxes.cls.tolist()  # Convert tensor to list
                xyxy = boxes.xyxy
                conf = boxes.conf
                xywh = boxes.xywh  # box with xywh format, (N, 4)
                for class_index in cls:
                    class_name = class_names[int(class_index)]
                    #print("Class:", class_name)

            pred_cls = np.array(cls)
            conf = conf.detach().cpu().numpy()
            xyxy = xyxy.detach().cpu().numpy()
            bboxes_xywh = xywh
            bboxes_xywh = xywh.cpu().numpy()
            bboxes_xywh = np.array(bboxes_xywh, dtype=float)
            
            if len(bboxes_xywh) > 0:
                tracks = tracker.update(bboxes_xywh, conf, og_frame)
                
                for track in tracker.tracker.tracks:
                    track_id = track.track_id
                    hits = track.hits
                    x1, y1, x2, y2 = track.to_tlbr()  # Get bounding box coordinates in (x1, y1, x2, y2) format
                    w = x2 - x1  # Calculate width
                    h = y2 - y1  # Calculate height

                    # Set color values for red, blue, and green
                    red_color = (0, 0, 255)  # (B, G, R)
                    blue_color = (255, 0, 0)  # (B, G, R)
                    green_color = (0, 255, 0)  # (B, G, R)

                    # Determine color based on track_id
                    color_id = track_id % 3
                    if color_id == 0:
                        color = red_color
                    elif color_id == 1:
                        color = blue_color
                    else:
                        color = green_color

                    cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x1 + w), int(y1 + h)), color, 2)

                    text_color = (0, 0, 0)  # Black color for text
                    cv2.putText(og_frame, f"{class_name}-{track_id}", (int(x1) + 10, int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

                    # Add the track_id to the set of unique track IDs
                    unique_track_ids.add(track_id)

            # Update the fire, smoke count and unique track IDs
            fire_count = sum([1 for c in pred_cls if class_names[int(c)] == 'fire'])
            smoke_count = sum([1 for c in pred_cls if class_names[int(c)] == 'smoke'])
            
            # Update FPS and place on frame
            current_time = perf_counter()
            elapsed = (current_time - start_time)
            counter += 1
            if elapsed > 1:
                fps = counter / elapsed
                counter = 0
                start_time = current_time

            # Draw fire and smoke count on the frame
            cv2.putText(og_frame, f"Fire: {fire_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            
            cv2.putText(og_frame, f"Smoke: {smoke_count}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

            # Append the frame to the list
            frames.append(og_frame)

            # Write the frame to the output video file
            out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))
        
        else:
            break
        
    cap.release()
    out.release()

In [18]:
# Apply DeepSort to detect and tracking human
if __name__ == "__main__":
    track_object(r"vid/location", r'vid/location/video.mp4') # input_vid, save_path

{0: 'fire', 1: 'smoke'}



0: 384x640 (no detections), 24.0ms
Speed: 1.0ms preprocess, 24.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.0ms
Speed: 1.0ms preprocess, 20.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.0ms
Speed: 1.0ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.0ms
Speed: 1.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 23.0ms
Speed: 1.0ms preprocess, 23.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 23.5ms
Speed: 1.0ms preprocess, 23.5ms i

KeyboardInterrupt: 

### Training Process

In [12]:
# Train the model on each dataset If. pretrained, add pretrained=weights.pt to the command above
# If you got some problem with the training like data.yaml not found, check the data folder and replace the train,test,val path with ../train,../test,../val
!yolo task=detect mode=train model=yolov8x.pt imgsz=640 data=C:/Work/Fire-Guardian/firedetectyolov7-2/data.yaml epochs=30 batch=16 name=data1 patience=10 plots=True val=True verbose=True  device=0

Ultralytics YOLOv8.2.2 🚀 Python-3.12.2 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)


train: Scanning C:\Work\Fire-Guardian\firedetectyolov7-2\train\labels.cache... 6008 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6008/6008 [00:00<?, ?it/s]
train: Scanning C:\Work\Fire-Guardian\firedetectyolov7-2\train\labels.cache... 6008 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6008/6008 [00:00<?, ?it/s]

val: Scanning C:\Work\Fire-Guardian\firedetectyolov7-2\valid\labels.cache... 1212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1212/1212 [00:00<?, ?it/s]
val: Scanning C:\Work\Fire-Guardian\firedetectyolov7-2\valid\labels.cache... 1212 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1212/1212 [00:00<?, ?it/s]

  0%|          | 0/376 [00:00<?, ?it/s]
       1/30      12.7G      2.487      4.093      1.999         88        640:   0%|          | 0/376 [00:02<?, ?it/s]
       1/30      12.7G      2.487      4.093      1.999         88        640:   0%|          | 1/376 [00:02<17:12,  2.75s/it]
       1/30      12.7G      2.281      3.826      2.099         59 


engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=C:/Work/Fire-Guardian/firedetectyolov7-2/data.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=data1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, opti


      21/30      13.4G      1.583      1.368      1.629         43        640:  11%|█         | 42/376 [00:22<02:44,  2.03it/s]
      21/30      13.4G      1.583      1.368      1.629         43        640:  11%|█▏        | 43/376 [00:22<02:47,  1.99it/s]
      21/30      13.4G      1.585      1.365      1.632         28        640:  11%|█▏        | 43/376 [00:23<02:47,  1.99it/s]
      21/30      13.4G      1.585      1.365      1.632         28        640:  12%|█▏        | 44/376 [00:23<02:44,  2.01it/s]
      21/30      13.4G      1.585      1.368      1.633         34        640:  12%|█▏        | 44/376 [00:23<02:44,  2.01it/s]
      21/30      13.4G      1.585      1.368      1.633         34        640:  12%|█▏        | 45/376 [00:23<02:44,  2.01it/s]
      21/30      13.4G      1.578      1.363      1.629         26        640:  12%|█▏        | 45/376 [00:24<02:44,  2.01it/s]
      21/30      13.4G      1.578      1.363      1.629         26        640:  12%|█▏        | 46/376 

### Validation Process

In [10]:
# Test the model on fire4 dataset
!yolo task=detect mode=val model=best/weight/location data={fire3_dataset.location}/data.yaml imgsz=640 verbose=True 

Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs

val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels...:   0%|          | 0/11205 [00:00<?, ?it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 33 images, 0 backgrounds, 0 corrupt:   0%|          | 33/11205 [00:00<00:38, 292.90it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 97 images, 0 backgrounds, 0 corrupt:   1%|          | 97/11205 [00:00<00:24, 461.97it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 161 images, 0 backgrounds, 0 corrupt:   1%|▏         | 161/11205 [00:00<00:21, 513.68it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels... 217 images, 0 backgrounds, 0 corrupt:   2%|▏         | 217/11205 [00:00<00:21, 521.95it/s]
val: Scanning D:\Fire-Alarm-Detection\Wildfire-V2-5\valid\labels

### Test Process

In [32]:
# Test predict on video, in here I use a video in check folder
!yolo task=detect mode=predict model=best/weight/location source=data/to/test/the/model imgsz=640

Ultralytics YOLOv8.0.196  Python-3.11.7 torch-2.2.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs

video 1/1 (1/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x640 1 smoke, 127.6ms
video 1/1 (2/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x640 1 smoke, 19.0ms
video 1/1 (3/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x640 1 smoke, 19.0ms
video 1/1 (4/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x640 1 smoke, 19.0ms
video 1/1 (5/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x640 1 smoke, 18.5ms
video 1/1 (6/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x640 1 smoke, 19.0ms
video 1/1 (7/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x640 1 smoke, 18.7ms
video 1/1 (8/719) D:\Fire-Alarm-Detection\check\Set_2\train_videos\smoke5.mp4: 384x64